In [1]:
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
producers = nx.read_gml("./networks/producers_network.gml")
themes = nx.read_gml("./networks/themes_network.gml")

In [3]:
C_b_p = nx.betweenness_centrality(producers)
C_c_p = nx.closeness_centrality(producers)
C_d_p = nx.degree_centrality(producers)
clustering_p = nx.clustering(producers)
C_b_t = nx.betweenness_centrality(themes)
C_c_t = nx.closeness_centrality(themes)
C_d_t = nx.degree_centrality(themes)
clustering_t = nx.clustering(themes)

In [4]:
animes = pd.read_csv("Anime.csv")
animes.dropna(subset=["Score"], inplace=True)
animes = animes.assign(Producers=animes["Producers"].str.split(", ")).explode("Producers")
animes = animes.assign(Themes=animes["Themes"].str.split(", ")).explode("Themes")
animes = animes[
    (animes["Producers"]!="Unknown") & 
    (animes["Producers"]!="Ltd.")
    ]
animes = animes[animes["Themes"]!="Unknown"]
animes = animes.reset_index().drop(columns=["index"])

In [13]:
data = animes[["ID", "Title", "Producers", "Themes", "Score", "Episodes", "Duration_Minutes", "Rating", "Popularity"]].copy()
data = pd.get_dummies(data, columns = ["Rating"])
data[
    ["Producers_closeness_centrality",
    "Producers_betweenness_centrality",
    "Producers_degree_centrality",
    "Producers_clustering_coefficient",
    "Themes_closeness_centrality",
    "Themes_betweenness_centrality",
    "Themes_degree_centrality",
    "Themes_clustering_coefficient"]] = np.nan
for i in range(len(data)):
    producer = data.loc[i,"Producers"]
    theme = data.loc[i,"Themes"]
    data.loc[i,"Producers_closeness_centrality"] = C_c_p[producer]
    data.loc[i,"Producers_betweenness_centrality"] = C_b_p[producer]
    data.loc[i,"Producers_degree_centrality"] = C_d_p[producer]
    data.loc[i,"Producers_clustering_coefficient"] = clustering_p[producer]
    data.loc[i,"Themes_closeness_centrality"] = C_c_t[theme]
    data.loc[i,"Themes_betweenness_centrality"] = C_b_t[theme]
    data.loc[i,"Themes_degree_centrality"] = C_d_t[theme]
    data.loc[i,"Themes_clustering_coefficient"] = clustering_t[theme]
data.dropna(inplace=True)
data.head()

,ID,Title,Producers,Themes,Score,Episodes,Duration_Minutes,Popularity,Rating_G - All Ages,Rating_None,...,Rating_R+ - Mild Nudity,Rating_Rx - Hentai,Producers_closeness_centrality,Producers_betweenness_centrality,Producers_degree_centrality,Producers_clustering_coefficient,Themes_closeness_centrality,Themes_betweenness_centrality,Themes_degree_centrality,Themes_clustering_coefficient
0,16498,Shingeki no Kyojin,Production I.G,Gore,8.531,25.0,24.0,1,0,0,...,0,0,0.363724,0.004022,0.058783,0.324487,0.636364,0.007484,0.428571,0.661905
1,16498,Shingeki no Kyojin,Production I.G,Military,8.531,25.0,24.0,1,0,0,...,0,0,0.363724,0.004022,0.058783,0.324487,0.720588,0.018642,0.612245,0.597701
2,16498,Shingeki no Kyojin,Production I.G,Survival,8.531,25.0,24.0,1,0,0,...,0,0,0.363724,0.004022,0.058783,0.324487,0.569767,0.001482,0.244898,0.757576
3,16498,Shingeki no Kyojin,Dentsu,Gore,8.531,25.0,24.0,1,0,0,...,0,0,0.415496,0.024174,0.181135,0.151915,0.636364,0.007484,0.428571,0.661905
4,16498,Shingeki no Kyojin,Dentsu,Military,8.531,25.0,24.0,1,0,0,...,0,0,0.415496,0.024174,0.181135,0.151915,0.720588,0.018642,0.612245,0.597701


In [14]:
data = data.groupby("ID").agg(np.average).reset_index()

/Users/bennycjy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


In [15]:
data.shape

(5455, 20)

In [17]:
data.to_csv("data/processed_anime.csv", index=False)